# 📓 Spec-Driven Development (SDD) 실습 예제

---
## 0. 소개
이 노트북은 **OpenAPI 스펙을 기반으로 FastAPI 애플리케이션을 자동 생성하고 검증**하는 과정을 다룹니다.  
핵심 메시지: **"코드는 스펙을 따른다 (Code Follows Spec)"**

## 1. OpenAPI 스펙 작성
먼저 간단한 **사용자(User) API**를 정의합니다.

In [11]:
%%writefile api-spec.yaml
openapi: 3.0.3
info:
  title: User Service API
  version: 1.0.0
paths:
  /users:
    get:
      summary: Get list of users
      responses:
        '200':
          description: A JSON array of users
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/User'
    post:
      summary: Create a new user
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/UserCreate'
      responses:
        '201':
          description: User created
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/User'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
components:
  schemas:
    User:
      type: object
      properties:
        id:
          type: integer
        name:
          type: string
    UserCreate:
      type: object
      properties:
        name:
          type: string
      required:
        - name

Overwriting api-spec.yaml


## 2. 스펙 기반 모델 코드 생성
`datamodel-code-generator` 라이브러리를 사용해 Pydantic 모델을 생성합니다.

In [3]:
!pip install datamodel-code-generator

  Using cached black-25.1.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (81 kB)
  Using cached isort-6.0.1-py3-none-any.whl.metadata (11 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
Using cached isort-6.0.1-py3-none-any.whl (94 kB)
Using cached black-25.1.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl (1.8 MB)
Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [datamodel-code-generator]el-code-generator]


In [4]:
!datamodel-codegen --input api-spec.yaml --input-file-type openapi --output models.py

## 3. FastAPI 서버 뼈대 작성
스펙을 기반으로 FastAPI 서버를 구현합니다.

In [12]:
%%writefile main.py
from fastapi import FastAPI
from models import User, UserCreate

app = FastAPI()

@app.get("/users", response_model=list[User])
def list_users():
    return [User(id=1, name="Alice")]

@app.post("/users", response_model=User, status_code=201)
def create_user(user: UserCreate):
    return User(id=2, name=user.name)

Overwriting main.py


## 4. 서버 실행 및 확인
Jupyter 노트북에서 uvicorn을 실행합니다. (백그라운드 모드)

In [6]:
!pip install fastapi uvicorn

  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
Using cached fastapi-0.116.1-py3-none-any.whl (95 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastapi]


In [13]:
import subprocess, time

# 서버 실행 (백그라운드)
process = subprocess.Popen(
    ["uvicorn", "main:app", "--port", "8000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(3)  # 서버 기동 대기

## 5. 스펙 기반 테스트 자동화
`schemathesis`를 사용해 OpenAPI 스펙과 실제 서버 구현이 일치하는지 자동 검증합니다.

In [8]:
!pip install schemathesis

  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached tomli-2.2.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached rfc3339_validator-0.1.4-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached iniconfig-2.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached types_python_dateutil-2.9.0.20250822-py3-none-any.whl.metadata (1.8 kB)
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.7/533.7 kB 11.1 MB/s  0:00:00
Using

In [15]:
!schemathesis run api-spec.yaml --url=http://localhost:8000

Schemathesis v4.1.4
━━━━━━━━━━━━━━━━━━━

 🕐  Loading specification from api-spec.yaml0m
 ✅  Loaded specification from api-spec.yaml (in 0.15s)                         

     Base URL:         http://localhost:8000                                    
     Specification:    Open API 3.0.3                                           
     Operations:       2 selected / 2 total                                     

 🕛  Probing API capabilities0m
 ✅  API capabilities:                                                          

     Supports NULL byte in headers:    ✘                                        

     Examples

     0:00:00 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% (2/2)


     Examples

     0:00:00 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% (2/2)


     🚫 1 error  ⏭  1 skipped
 🚫  Examples (in 0.11s)                                                        
                                                                                
     🚫 

## 6. 서버 종료

In [16]:
process.terminate()

# 📌 정리
1. OpenAPI 스펙을 작성한다.  
2. 모델/클라이언트/문서/테스트는 스펙에서 자동 생성한다.  
3. FastAPI는 스펙에 정의된 API만 구현하면 된다.  
4. **CI/CD 파이프라인에 `schemathesis`를 추가하면, 스펙-코드 일관성이 자동 보장된다.**